In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np


def recall_at_k(y_true, y_prob, k=0.1):
    """
    Tahmin edilen olasılıkların en üst k%'sını pozitif etiketleyerek recall değerini hesaplar.

    Parametreler:
        y_true (list): Gerçek ikili etiketler.
        y_prob (list): Tahmin edilen olasılıklar.
        k (float): Pozitif etiketlenecek olasılıkların yüzdelik dilimi (varsayılan 0.1).

    Döndürür:
        float: En iyi k% tahminlerindeki recall oranı.
    """
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    n = len(y_true)
    m = max(1, int(np.round(k * n)))
    order = np.argsort(-y_prob, kind="mergesort")
    top = order[:m]

    tp_at_k = y_true[top].sum()
    P = y_true.sum()

    return float(tp_at_k / P) if P > 0 else 0.0


def lift_at_k(y_true, y_prob, k=0.1):
    """
    Tahmin edilen olasılıkların en üst k%'sını pozitif etiketleyerek lift (precision/prevalence) değerini hesaplar.

    Parametreler:
        y_true (list): Gerçek ikili etiketler.
        y_prob (list): Tahmin edilen olasılıklar.
        k (float): Pozitif etiketlenecek olasılıkların yüzdelik dilimi (varsayılan 0.1).

    Döndürür:
        float: En iyi k% tahminlerindeki lift değeri.
    """
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    n = len(y_true)
    m = max(1, int(np.round(k * n)))
    order = np.argsort(-y_prob, kind="mergesort")
    top = order[:m]

    tp_at_k = y_true[top].sum()
    precision_at_k = tp_at_k / m
    prevalence = y_true.mean()

    return float(precision_at_k / prevalence) if prevalence > 0 else 0.0


def convert_auc_to_gini(auc):
    """
    ROC AUC skorunu Gini katsayısına dönüştürür.

    Gini katsayısı, ROC AUC skorunun doğrusal bir dönüşümüdür.

    Parametreler:
        auc (float): ROC AUC skoru (0 ile 1 arasında).

    Döndürür:
        float: Gini katsayısı (-1 ile 1 arasında).
    """
    return 2 * auc - 1


def ing_hubs_datathon_metric(y_true, y_prob):
    """
    Gini, recall@10% ve lift@10% metriklerini birleştiren özel bir metrik hesaplar.

    Metrik, her bir skoru bir baseline modelin metrik değerlerine göre oranlar ve aşağıdaki ağırlıkları uygular:
    - Gini: %40
    - Recall@10%: %30
    - Lift@10%: %30

    Parametreler:
        y_true (list): Gerçek ikili etiketler.
        y_prob (list): Tahmin edilen olasılıklar.

    Döndürür:
        float: Ağırlıklandırılmış bileşik skor.
    """
    # final metrik için ağırlıklar
    score_weights = {
        "gini": 0.4,
        "recall_at_10perc": 0.3,
        "lift_at_10perc": 0.3,
    }

    # baseline modelin her bir metrik için değerleri
    baseline_scores = {
        "roc_auc": 0.6925726757936908,
        "recall_at_10perc": 0.18469015795868773,
        "lift_at_10perc": 1.847159286784029,
    }

    # y_prob tahminleri için metriklerin hesaplanması
    roc_auc = roc_auc_score(y_true, y_prob)
    recall_at_10perc = recall_at_k(y_true, y_prob, k=0.1)
    lift_at_10perc = lift_at_k(y_true, y_prob, k=0.1)

    new_scores = {
        "roc_auc": roc_auc,
        "recall_at_10perc": recall_at_10perc,
        "lift_at_10perc": lift_at_10perc,
    }

    # roc auc değerlerinin gini değerine dönüştürülmesi
    baseline_scores["gini"] = convert_auc_to_gini(baseline_scores["roc_auc"])
    new_scores["gini"] = convert_auc_to_gini(new_scores["roc_auc"])

    # baseline modeline oranlama
    final_gini_score = new_scores["gini"] / baseline_scores["gini"]
    final_recall_score = new_scores["recall_at_10perc"] / baseline_scores["recall_at_10perc"]
    final_lift_score = new_scores["lift_at_10perc"] / baseline_scores["lift_at_10perc"]

    # ağırlıklandırılmış metriğin hesaplanması
    final_score = (
        final_gini_score * score_weights["gini"] +
        final_recall_score * score_weights["recall_at_10perc"] + 
        final_lift_score * score_weights["lift_at_10perc"]
    )
    return final_score


In [2]:
import pandas as pd
import numpy as np
train_data_final = pd.read_parquet('/kaggle/input/aaa222/train_data_v7.parquet')
test_data_final = pd.read_parquet('/kaggle/input/aaa222/test_data_v7.parquet')

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import lightgbm as lgb
from catboost import CatBoostClassifier

target_col = 'churn'
cat_features = ['gender', 'work_type', 'province', 'religion', 'work_sector']

df = train_data_final.copy()
X_test_orig = test_data_final.copy()
X_orig = df.drop(target_col, axis=1)
y = df[target_col]
#5 fold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

lgb_params = {
    'n_estimators': 1000,
    'learning_rate': 0.1,
    'max_depth': 6,
    'reg_lambda': 3,
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'random_state': 42,
    'class_weight': 'balanced',
    'verbosity': -1
}

catboost_params = {
    'iterations': 1000,
    'depth': 6,
    'learning_rate': 0.1,
    'l2_leaf_reg': 3,
    'border_count': 128,
    'random_seed': 42,
    'eval_metric': 'AUC',
    'verbose': 0,
    'auto_class_weights': 'Balanced'
}

#model baslangıcı
models = {
    "CatBoost": ("catboost", catboost_params),
    "LightGBM": ("lgbm", lgb_params),
}
#oof
oof_preds = {name: np.zeros(len(X_orig)) for name in models.keys()}
fold_test_preds_dict = {name: [] for name in models.keys()}
for model_name, (model_type, params) in models.items():
    print(model_name)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_orig, y), 1):
        X_train, X_val = X_orig.iloc[train_idx].copy(), X_orig.iloc[val_idx].copy()
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        if model_type == "catboost":
            model = CatBoostClassifier(**params)
            model.fit(X_train, y_train, cat_features=cat_features,
                      eval_set=(X_val, y_val), early_stopping_rounds=200, verbose=0)
        elif model_type == "lgbm":
            model = lgb.LGBMClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                      categorical_feature=cat_features,
                      callbacks=[lgb.early_stopping(200, verbose=False)])
        y_val_pred = model.predict_proba(X_val)[:, 1]
        oof_preds[model_name][val_idx] = y_val_pred

        test_pred = model.predict_proba(X_test_orig)[:, 1]
        fold_test_preds_dict[model_name].append(test_pred)

        auc = roc_auc_score(y_val, y_val_pred)
        custom_score = ing_hubs_datathon_metric(y_val, y_val_pred)
        print(f"Fold {fold} - AUC: {auc:.4f} | Custom: {custom_score:.4f}")

base_model_test_preds = {name: np.mean(preds, axis=0) 
                         for name, preds in fold_test_preds_dict.items()}
meta_X = np.column_stack([oof_preds[name] for name in models.keys()])
meta_test_X = np.column_stack([base_model_test_preds[name] for name in models.keys()])

meta_oof_preds = np.zeros(len(y))
meta_test_preds_per_fold = []

for fold, (train_idx, val_idx) in enumerate(kf.split(meta_X, y), 1):
    X_train_meta, X_val_meta = meta_X[train_idx], meta_X[val_idx]
    y_train_meta, y_val_meta = y.iloc[train_idx], y.iloc[val_idx]

    meta_model = LogisticRegression(max_iter=1000)
    meta_model.fit(X_train_meta, y_train_meta)

    y_val_meta_pred = meta_model.predict_proba(X_val_meta)[:, 1]
    meta_oof_preds[val_idx] = y_val_meta_pred
    meta_test_preds_per_fold.append(meta_model.predict_proba(meta_test_X)[:, 1])

meta_test_preds = np.mean(meta_test_preds_per_fold, axis=0)
#sonuc
final_auc = roc_auc_score(y, meta_oof_preds)
print(f"\n✅ Meta-classifier 5-Fold OOF AUC: {final_auc:.4f}")
print("Meta-classifier test tahminleri hazır.")




================= CatBoost =================
Fold 1 - AUC: 0.7236 | Custom: 1.1936
Fold 2 - AUC: 0.7197 | Custom: 1.1485
Fold 3 - AUC: 0.7156 | Custom: 1.1605
Fold 4 - AUC: 0.7208 | Custom: 1.1557
Fold 5 - AUC: 0.7252 | Custom: 1.1942


================= LightGBM =================
Fold 1 - AUC: 0.7234 | Custom: 1.1947
Fold 2 - AUC: 0.7150 | Custom: 1.1454
Fold 3 - AUC: 0.7137 | Custom: 1.1351
Fold 4 - AUC: 0.7191 | Custom: 1.1694
Fold 5 - AUC: 0.7225 | Custom: 1.1895

✅ Meta-classifier 5-Fold OOF AUC: 0.7217
Meta-classifier test tahminleri hazır.
